In [7]:
from bs4 import BeautifulSoup

import requests
import pandas as pd
import random
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id = "ed6ca9a8672b466ea3c61d928303c023", 
    client_secret = "67b211c4b3bd4dc89432568968dcee41"))

In [8]:
# function that creates a soup
def create_soup(html):
    url = html
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

soup = create_soup("https://www.billboard.com/charts/hot-100")

# function that creates a list
def create_list(part_html, soup):
    list_name = []
    for element in soup.select(part_html):
        list_name.append(element.get_text())
    return list_name

songs = create_list(".chart-element__information__song", soup)
artists = create_list(".chart-element__information__artist", soup)
rankLW = create_list("span.chart-element__meta.text--center.color--secondary.text--last", soup)
peaks = create_list("span.chart-element__meta.text--center.color--secondary.text--peak", soup)
wocs = create_list("span.chart-element__meta.text--center.color--secondary.text--week", soup)

# create a dataframe of the lists
dftop100 = pd.DataFrame({
    "Song": songs,
    "Artist": artists,
    "Rank Last Week": rankLW,
    "Peak": peaks,
    "Weeks on Chart": wocs
})

# create list with lower cases of the songs
low_top100 = []
for song in songs:
    low_top100.append(song.lower())
    
# function to give random hot song
def song_check():
    input_song = str(input("Choose a song: ")).lower()
    
# Check if song is 'hot'.
# ADD THE ARTIST OF THE SONG
    if input_song in low_top100:
        print("Nice choice! This song is 'hot'.")
        random_song = random.sample(songs, k = 1)[0]
        print("I think you will also like:", random_song)
        spot_chosen_song = random_song
        input_like_song = str(input("Do you like it? (Yes/No) ").lower())
        while input_like_song == "no":
            print("Maybe you like:", random.sample(songs, k = 1)[0])
            input_like_song = str(input("Do you like it? (Yes/No) ").lower())
        if input_like_song == "yes":      
            print("Glad to help!")
            spot_chosen_song = "fuck"
            
        
# If song is not 'hot'.
# Check first 3 searches in spotify which one you mean
    elif input_song not in low_top100:
        print("This song is not 'hot'... Let me search for it in Spotify.")
        
        spotify_input = spotify.search(q = input_song, limit = 3)

        try:
            spot1 = spotify_input["tracks"]["items"][0]["name"]
            spot2 = spotify_input["tracks"]["items"][1]["name"] 
            spot3 = spotify_input["tracks"]["items"][2]["name"]
            spotart1 = spotify_input["tracks"]["items"][0]["artists"][0]["name"]
            spotart2 = spotify_input["tracks"]["items"][1]["artists"][0]["name"]
            spotart3 = spotify_input["tracks"]["items"][2]["artists"][0]["name"]

            spotsong_input1 = str(input(print("Do you mean the song:", 
                                    spot1, "\nFrom:", spotart1, "? (Yes/No)"))).lower()
            
            if spotsong_input1 == "yes":
                spot_chosen_song = spot1
            elif spotsong_input1 != "yes":
                spotsong_input2 = str(input(print("Do you mean the song:", 
                                        spot2, "\nFrom:", spotart2, "? (Yes/No)"))).lower()

                if spotsong_input2 == "yes":
                    spot_chosen_song = spot2
                elif spotsong_input2 != "yes":
                    spotsong_input3 = str(input(print("Do you mean the song:", 
                                            spot3, "\nFrom:", spotart3, "? (Yes/No)"))).lower()

                    if spotsong_input3 == "yes":
                        spot_chosen_song = spot3
                    elif spotsong_input3 != "yes":
                        print("I'm sorry.. Then I can not help you.")
        except:
            print("I'm sorry, there are no results for this in spotify.. \nTry again.")
            spot_chosen_song = ''

    return spot_chosen_song

spot_chosen_song = song_check() 

if spot_chosen_song != "fuck":

    # Scale the dataframe of the playlist
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(df_features.select_dtypes(exclude = object))
    kmeans = KMeans(n_init=100)
    kmeans.fit(features_scaled)
    clusters = kmeans.predict(features_scaled)

    # Scale the audio features from input song
    input_features = spotify.audio_features(spotify.search(spot_chosen_song)["tracks"]["items"][0]["uri"])
    input_dfdummy = pd.DataFrame(input_features[0], index = range(len(input_features)))
    input_df = input_dfdummy.select_dtypes(exclude = object)
    scaled_input = scaler.transform(input_df)

    # In what cluster is the input song?
    input_cluster = kmeans.predict(scaled_input)[0]


    # Create lists cointaining the songs from every cluster
    df_clusters = df_features.copy()
    df_clusters["Cluster"] = clusters

    complete_input_cluster = df_clusters[df_clusters['Cluster'] == input_cluster]
    random_input_cluster = complete_input_cluster["Song"].sample(1)
    print("I think you will like:", random_input_cluster)
    print("Preview:", spotify.search(random_input_cluster)["tracks"]["items"][0]["preview_url"])

    # Check if user likes recommendation
    like_recommend = str(input("Do you like it? (Yes/No) ").lower())
    while like_recommend == "no":
        new_recommend = complete_input_cluster["Song"].sample(1)
        print("Maybe you like:", new_recommend)
        print("Preview:", spotify.search(new_recommend)["tracks"]["items"][0]["preview_url"])
        like_recommend = str(input("Do you like it? (Yes/No) ").lower())

    if like_recommend == "yes":
        print("Your music taste is shit.. But glad to help!")



Choose a song: mood
Nice choice! This song is 'hot'.
I think you will also like: Relacion
Do you like it? (Yes/No) no
Maybe you like: I Hope
Do you like it? (Yes/No) yes
Glad to help!


In [269]:
# !!!!!!!!!!!! DO NOT RUN AGAIN !!!!!!!!!!! DO NOT RUN !!!!!!!!!!!!!!

popular_names = []
popular_artists = []
popular_uri = []
popular_features = []


for item in popular_playlist:
    popular_names.append(item["track"]["name"])
    popular_uri.append(item["track"]["uri"])

for item in popular_playlist:
    for artist in item["track"]["artists"]:
        popular_artists.append(artist["name"])

for item in popular_playlist:
        popular_features.append(spotify.audio_features(item["track"]["uri"]))
        

In [10]:
# Get all of the songs of a playlist
def get_playlist_tracks():
    popular_playlist = []
    results = spotify.user_playlist_tracks(user = "spotify", playlist_id = "spotify:user:zurxor:playlist:4o8NBsWreC3OnKePUYw0dg")
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        popular_playlist.extend(results['items'])
    return popular_playlist

popular_playlist = get_playlist_tracks()



In [50]:
# Append features of the songs of the playlist in lists
danceability = []
key = []
energy = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration = []
time = []
    
for feature in popular_features:
    danceability.append(feature[0]["danceability"])
    key.append(feature[0]["key"])
    energy.append(feature[0]["energy"])
    loudness.append(feature[0]["loudness"])
    mode.append(feature[0]["mode"])
    speechiness.append(feature[0]["speechiness"])
    acousticness.append(feature[0]["acousticness"])
    instrumentalness.append(feature[0]["instrumentalness"])
    liveness.append(feature[0]["liveness"])
    valence.append(feature[0]["valence"])
    tempo.append(feature[0]["tempo"])
    duration.append(feature[0]["duration_ms"])
    time.append(feature[0]["time_signature"])

# Create a Dataframe of of the features of all songs from the playlist
df_features = pd.DataFrame({
    "Song": popular_names,
    "URI": popular_uri,
    "Danceability": danceability,
    "Key": key,
    "Energy": energy,
    "Loudness": loudness,
    "Mode": mode,
    "Speechiness": speechiness,
    "Acousticness": acousticness,
    "Instrumentalness": instrumentalness,
    "Liveness": liveness,
    "Valence": valence,
    "Tempo": tempo,
    "Duration": duration,
    "Time": time
    
})

NameError: name 'popular_features' is not defined